<img style="float:left" width="70%" src="pics/escudo_COLOR_1L_DCHA.png">
<img style="float:right" width="15%" src="pics/PythonLogo.svg">
<br style="clear:both;">


# Otelo - Práctica 1
### *Sistemas Inteligentes* (Curso 2022-2023)



<h2 style="display: inline-block; padding: 4mm; padding-left: 2em; background-color: navy; line-height: 1.3em; color: white; border-radius: 10px;">Hacia la versión "automática" del juego </h2>

## Docentes

 - Pedro Latorre Carmona
 
## Alumno

 - Roberto Lastras Santos

## Definición de la clase

A continuación, tenéis una versión completa al $80\%$ de la clase **Tablerootelo**.

In [ ]:
from copy import deepcopy
from typing import Tuple, List
from IPython.display import display
from ipywidgets import HTML, Dropdown ,Button, HBox, VBox, Box
import math
import time

In [ ]:
class Tablerootelo:
    
    def __init__(self, matrix):
        self.setMatrix(matrix)
    
    def __eq__(self, other) -> bool:
        
        for i in range(8):
            for j in range(8):
                if self.matrix[i][j] != other.matrix[i][j]:
                    return False
        return True
    
    def setMatrix(self, matrix):
        self.matrix = deepcopy(matrix) 
    
    def getMatrix(self) -> List[List]:
        return deepcopy(self.matrix)
    
    def placeTile(self, row: int, col: int, tile: int):
        self.matrix[row-1][col-1] = tile
    
    def utility(self) -> int:
        
        global utilidad
        
        all = [item for sublist in self.matrix for item in sublist]
        white = sum(1 for tile in all if tile == 2)
        black = sum(1 for tile in all if tile == 1)
                
        if white > black:
            diff = int((white / (black + white)) * 100)     
        else:
            diff = - int((black / (black + white)) * 100) 
    
        if self.moveCanBeMade(1) + self.moveCanBeMade(2) == 0:
            mobility = 0    
        else:
            mobility = 100 * self.moveCanBeMade(2) / (self.moveCanBeMade(2) + self.moveCanBeMade(1))
            
        utilidad = diff + mobility
        
        return utilidad

    def isAvailable(self, row: int, col: int, playerID: int, PLAYMODE:bool) -> int: 
  
        global changed
        global player
        global debug
        global victory
        global whiteTiles
        global blackTiles

        if PLAYMODE:
            self.matrix[row][col] = player
        count = 0
        __column = self.matrix[row]                        
        __row = [self.matrix[i][col] for i in range(0, 8)] 
 
        if playerID in __column[:col]:       
                                         
            changes = []
            searchCompleted = False             
            for i in range(col - 1, -1, -1):    
                if searchCompleted:
                    continue
                piece = __column[i]
                if piece == 0:        
                    changes = []
                    searchCompleted = True          
                elif piece == playerID:
                    searchCompleted = True
                else:
                    changes.append(i)  
                                  
            if searchCompleted:       
                                   
                count += len(changes)
                if PLAYMODE:
                    for i in changes:
                        self.matrix[row][i] = player

        if playerID in __column[col:]:
            changes = []
            searchCompleted = False

            for i in range(col + 1, 8, 1):
                if searchCompleted:
                    continue
                piece = __column[i]
                if piece == 0:
                    changes = []
                    searchCompleted = True
                elif piece == playerID:
                    searchCompleted = True
                else:
                    changes.append(i)
                                           
            if searchCompleted:       
                                   
                count += len(changes)
                if PLAYMODE:
                    for i in changes:
                        self.matrix[row][i] = player  
                                   
        if playerID in __row[:row]:    
                                   
            changes = []
            searchCompleted = False

            for i in range(row - 1, -1, -1): 
                if searchCompleted:
                    continue
                piece = __row[i]
                if piece == 0:
                    changes = []
                    searchCompleted = True
                elif piece == playerID:
                    searchCompleted = True
                else:
                    changes.append(i)
            if searchCompleted:
                count += len(changes)
                if PLAYMODE:
                    for i in changes:
                        self.matrix[i][col] = player

        if playerID in __row[row:]:
            changes = []
            searchCompleted = False

            for i in range(row + 1, 8, 1): 
                if searchCompleted:
                    continue
                piece = __row[i]
                if piece == 0:
                    changes = []
                    searchCompleted = True
                elif piece == playerID:
                    searchCompleted = True
                else:
                    changes.append(i)
            if searchCompleted:
                count += len(changes)
                if PLAYMODE:
                    for i in changes:
                        self.matrix[i][col] = player

        i = 1    
        ulDiagonal = []
        while row - i >= 0 and col - i >= 0:
            ulDiagonal.append(self.matrix[row - i][col - i])
            i += 1
        if playerID in ulDiagonal:
            changes = []
            searchCompleted = False
            for i in range(0, len(ulDiagonal)):
                piece = ulDiagonal[i]
                if searchCompleted:
                    continue
                if piece == 0:
                    changes = []
                    searchCompleted = True
                elif piece == playerID:
                    searchCompleted = True
                else:
                    changes.append((row - (i + 1), col - (i + 1)))   
                                                                
            if searchCompleted:
                count += len(changes)
                if PLAYMODE:
                    for i, j in changes:
                        self.matrix[i][j] = player

        i = 1
        urDiagonal = []
        while row + i < 8 and col - i >= 0:
            urDiagonal.append(self.matrix[row + i][col - i])
            i += 1
        if playerID in urDiagonal:
            changes = []
            searchCompleted = False
            for i in range(0, len(urDiagonal)):
                piece = urDiagonal[i]
                if searchCompleted:
                    continue
                if piece == 0:
                    changes = []
                    searchCompleted = True
                elif piece == playerID:
                    searchCompleted = True
                else:
                    changes.append((row + (i + 1), col - (i + 1)))    
                                                                 
            if searchCompleted:
                count += len(changes)
                if PLAYMODE:
                    for i, j in changes:
                        self.matrix[i][j] = player

        i = 1
        llDiagonal = []
        while row - i >= 0 and col + i < 8:
            llDiagonal.append(self.matrix[row - i][col + i])
            i += 1
        if playerID in llDiagonal:
            changes = []
            searchCompleted = False

            for i in range(0, len(llDiagonal)):
                piece = llDiagonal[i]
                if searchCompleted:
                    continue
                if piece == 0:
                    changes = []
                    searchCompleted = True
                elif piece == playerID:
                    searchCompleted = True
                else:
                    changes.append((row - (i + 1), col + (i + 1)))    
                                                                 
            if searchCompleted:
                count += len(changes)
                if PLAYMODE:
                    for i, j in changes:
                        self.matrix[i][j] = player
                    
        i = 1
        lrDiagonal = []
        while row + i < 8 and col + i < 8:
            lrDiagonal.append(self.matrix[row + i][col + i])
            i += 1
        if playerID in lrDiagonal:
            changes = []
            searchCompleted = False

            for i in range(0, len(lrDiagonal)):
                piece = lrDiagonal[i]
                if searchCompleted:
                    continue
                if piece == 0:
                    changes = []
                    searchCompleted = True
                elif piece == playerID:
                    searchCompleted = True
                else:
                    changes.append((row + (i + 1), col + (i + 1)))                                                    

            if searchCompleted:
                count += len(changes)
                if PLAYMODE:
                    for i, j in changes:
                        self.matrix[i][j] = player

        if count == 0 and PLAYMODE:
            self.matrix[row][col] = 0          
    
        return count
    
    def moveCanBeMade(self, playerID:int) -> bool:
        movesFound = False
        for row in range(0, 8):
            for col in range(0, 8):
                if movesFound:
                    continue
                elif self.matrix[row][col] == 0:
                    numAvailableMoves = self.isAvailable(row, col, playerID, PLAYMODE=False)
                    if numAvailableMoves > 0:
                        movesFound = True
        return movesFound
    
    def endGame(self) -> int:
        global victory
        global changed
        global whiteTiles
        global blackTiles

        all = [item for sublist in self.matrix for item in sublist]
        white = sum(1 for tile in all if tile == 2)   
        black = sum(1 for tile in all if tile == 1)
    
        if white > black:
            victory = 2
        elif white < black:
            victory = 1
        else:
            victory = -1
        
        changed = True
        
        return victory
    
    def performMove(self,x:int, y:int):
        global changed
        global player
        global fin
        global invalido
        global no_move

        if self.matrix[x][y] != 0: 
            
            print("¡La celda ya está ocupada!")
            
        else:
            numFlipped = self.isAvailable(x, y, player, PLAYMODE=True) 
        
            if(numFlipped == 0 and player == 1):
                invalido = False
            else:

                print("¡Se dieron la vuelta " + str(numFlipped) + " fichas!")
                changed = True

                # check game ending
                allTiles = [item for sublist in self.matrix for item in sublist]
                emptyTiles = sum(1 for tile in allTiles if tile == 0)
                whiteTiles = sum(1 for tile in allTiles if tile == 2)
                blackTiles = sum(1 for tile in allTiles if tile == 1)

                print("Estado actual - Vacias: " + str(emptyTiles) + " Blancas: " + str(
                    whiteTiles) + " Negras: " + str(blackTiles))

                if whiteTiles < 1 or blackTiles < 1 or emptyTiles < 1:
                    fin = True
                    victoria = self.endGame()
                    return victoria

                movesFound = self.moveCanBeMade(3 - player)        

                if not movesFound:  

                    print("¡El jugador " + str(3 - player) + " no puede mover!")
                    movesFound = self.moveCanBeMade(player)

                    if not movesFound:
                        print("¡El jugador " + str(player) + " tampoco puede mover!")
                        victoria=self.endGame()
                        fin = True
                        no_move = 3
                        return victoria

                    else:
                        print("El jugador " + str(player) + " puede mover, asi que ¡adelante!")
                        #if useAI and player == 2: 
                        if player == 2: 

                            performMoveMinMax(self, player)

                        changed = True
                else:
                    player = 3 - player
                    changed = True
                    #fin = True
                    #no_move = 3


## Función asociada al método *Minimax* (con poda $\alpha-\beta$)

In [ ]:
# Función que realiza el algoritmo Minimax (con poda 𝛼−𝛽 )

def miniMax(state:Tablerootelo, currentLevel:int, maxLevel:int, player:int, alpha:int, beta:int, stop:bool) -> Tuple[Tablerootelo, int]:

    successorMatrices = []
    
    if (not state.moveCanBeMade(player) or currentLevel == maxLevel):
        return (state.matrix,state.utility())    

    for row in range(0, 8):
        for col in range(0, 8):
            if state.getMatrix()[row][col] == 0:
                
                # Llama al método "isAvailable" y con la información que te devuelva, sobre una copia de
                # la matriz original, hace el cambio correspondiente en [row][col]. Una vez generada esa versión cambiada
                # sobre la copia, hay que "anexarla" para considerarla como "hijo/a" (en "successorMatrices").
                
                casilla = state.isAvailable(row, col, player, False)
                if(casilla > 0):
                    state_sucesor = Tablerootelo(state.getMatrix())
                    state_sucesor.isAvailable(row, col, player, True)
                    successorMatrices.append(state_sucesor)
                
                
    if len(successorMatrices) == 0:
        stopDigging = True
        coste=state.utility()
        return (state.matrix, coste)
        
    bestMatrix = None

    if player == 2: # Parte MAX, de MINIMAX, con poda alfa-beta. 
        
        maxValue = -math.inf #alpha
        
        for hijo in successorMatrices:
            matriz, value = miniMax(hijo, currentLevel + 1, maxLevel, 1, alpha, beta, False)
            
            if(value > maxValue):
                maxValue = value
                bestMatrix = matriz
            """
            max(maxValue, value)
            """
            alpha = max(alpha, value)
            if beta <= alpha:
                break
        coste = maxValue
        
    else: # Parte MIN, de MINIMAX, con poda alfa-beta
        
        minValue = math.inf #beta
        
        for hijo in successorMatrices:
            matriz, value = miniMax(hijo, currentLevel + 1, maxLevel, 2, alpha, beta, False)
            if(value < minValue):
                minValue = value
                bestMatrix = matriz
            """
            minValue = min(minValue, value)
            """
            beta = min(beta, value)
            if beta <= alpha:
                break
        coste = minValue
        
    return (bestMatrix,coste)


## Generación de la función performMoveMinMax

In [ ]:
# Función que debe proponer la casilla (en coordenadas (row,column)) que el ordenador debe ocupar como su siguiente 
# MEJOR MOVIMIENTO

def performMoveMinMax(state:Tablerootelo, player:int):
    
    matriz, coste = miniMax(state, 0, 2, player, -math.inf, math.inf, True)

    for row in range(0, 8):
        for col in range(0, 8):
            if matriz[row][col] == player and state.getMatrix()[row][col] == 0 :
                Movoptimo = (row, col)    #Recibe la mejor posicion donde realizar el movimiento
    
    movimiento = Movoptimo
     
    return movimiento


## Funciones para pintar el Tablero en la terminal y en HTML

In [ ]:
# Funcion que inicializa el tablero del juego

def inicializarTablero():
    
    tablero = [ [0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 1, 2, 0, 0, 0],
                [0, 0, 0, 2, 1, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0]  ]
    
    pintarTablero(tablero)

    return tablero


# Funcion que imprime los bordes del tablero y las fichas que contiene, y muestra la interfaz del HTML generado

def pintarTablero(tablero):
    
    BORDE_H = '  +---+---+---+---+---+---+---+---+'
    BORDE_V = '  |   |   |   |   |   |   |   |   |'
 
    print("\n")
    print('    1   2   3   4   5   6   7   8')
    print(BORDE_H)

    for y in range(8):
        print(BORDE_V)
        print(y+1, end=' ')
        for x in range(8):
            print('| %s' % (tablero[x][y]), end=' ')
        print('|')
        print(BORDE_V)
        print(BORDE_H)


In [ ]:
# Funcion que genera el codigo HTML del tablero

def juegoHTML(tablero):
    
    alto, ancho = len(tablero), len(tablero[0])    
    cadenaHTML = '<table> <tr>'    
    caracter = '<td></td>'
    fondo = '<td><img src="./ImagenesCasillasOtelo/FondoRojo.png" alt="" border=0 height=50 width=50></img></td>'
    negras = '<td><img src="./ImagenesCasillasOtelo/CaraNegraFichaFondoRojo.png" alt="" border=0 height=50 width=50></img></td>'
    blancas = '<td><img src="./ImagenesCasillasOtelo/CaraBlancaFichaFondoRojo.png" alt="" border=0 height=50 width=50></img></td>'
    saltoLinea = "</tr>"
    
    for i in range(8):
        for j in range(8):
            if tablero[j][i] == 1 :
                caracter = negras
            elif tablero[j][i] == 2 :
                caracter = blancas
            else:
                caracter = fondo
                
            cadenaHTML += caracter
        cadenaHTML += saltoLinea        
        
    cadenaHTML += saltoLinea 
    
    return cadenaHTML

## Función "main" del juego

In [ ]:
# Funcion que inicia el juego, realiza los movimientos y muestra los resultados

def startGame():
    
    DIGITS1TO8 = '1 2 3 4 5 6 7 8'.split()
    global player
    global fin
    global invalido
    global no_move

    player = 1
    fin = False
    invalido = True
    resultado = -1
    no_move = 0

    print('\nEmpieza el juagador '+ str(player) +'.')

    matriz = inicializarTablero()
    state = Tablerootelo(matriz)
    
    #Genera la interfaz en HTML
    visor=HTML()
    ui=VBox(children=[visor])
    display(ui)
    visor.value = juegoHTML(matriz)
    
    while True:

        if (player == 1): #Movimiento del JUGADOR
                
            print('\nJugador '+ str(player) +':')

            move = input('Introduce tu movimiento [row][col] (p.e: 35) o escribe "salir" para finalizar el juego: ').lower()
            print("\n")
        
            if move == 'salir':
                break
            else:
                #Comprueba que los datos introcidos coincidan con los movimientos posibles (DIGITS1TO8)
                if len(move) == 2 and move[0] in DIGITS1TO8 and move[1] in DIGITS1TO8:
                    columna = int(move[0]) - 1
                    fila = int(move[1]) - 1
                    
                    resultado = state.performMove(fila, columna)
                    
                    if(invalido == False):
                        #En caso de introducir un mal movimiento, permite introducir otro movimiento
                        print("MOVIMIENTO INVALIDO - Introduzca otro movimiento:")
                        
                    matriz = state.getMatrix()

                else:
                    print('No existe esa opcion')
        
        else:
            #Movimiento del ORDENADOR
            
            print('\nJugador '+ str(player) +':')

            fila, columna = performMoveMinMax(state, player)

            print("Movimiento ordenador: "+ str(columna+1) + str(fila+1))

            resultado = state.performMove(fila, columna)

            matriz = state.getMatrix()

        #Imprime el tablero actualizado
        pintarTablero(matriz)
        visor.value = juegoHTML(matriz)
        
        # Espera 2 segundos en cada movimiento que se realice
        time.sleep(2)

        #Finaliza el juego
        if(fin == True):
            resultado = state.performMove(fila, columna)
            break
    
    # Comprueba las celdas al finalizar el programa
    allTiles = [item for sublist in state.matrix for item in sublist]
    emptyTiles = sum(1 for tile in allTiles if tile == 0)
    whiteTiles = sum(1 for tile in allTiles if tile == 2)
    blackTiles = sum(1 for tile in allTiles if tile == 1)

    if whiteTiles < blackTiles:
        resultado = 1
    elif whiteTiles > blackTiles:
        resultado = 2
    elif whiteTiles == 2 and blackTiles == 2:
        resultado = -1
    else:
        resultado = 0
    
    #Muestra los resultados de la partida
    if(resultado == 1):
        print("\nHA GANADO JUGADOR 1\n")
        print("¡ENHORABUENA!")
    elif(resultado == 2):
        print("\nHA GANADO JUGADOR 2")
    elif(resultado == 0):
        print("\nHABEIS EMPATADO")
    elif(no_move == 3):
        print("\nFIN JUEGO")
    else:
        print("Gracias por jugar!")


## Inicio de Juego

In [ ]:
print('\n¡¡ BIENVENIDO A OTELO !!') 
startGame()